<a href="https://colab.research.google.com/github/shinjangwoon/jangwoonshin/blob/practice/%EA%B8%88%EC%9C%B5_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D(%ED%80%80%ED%8A%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
'''
 레벨-초급 스타일-밸류+퀄리티 
기대CAGR-15% 
종목개수-20~30개 
매수전략-PER 10 이하(종목이 충분할 경우 PER 5이하로 제한), 부재비율 50% 이하
'''


# Basic setting

##에프앤가이드

In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 스냅샷 URL
SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
# 재무비율 URL
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

In [ ]:
print(SNAP_URL)
print(RATIO_URL)

In [ ]:
# formatting 예시
#'AB%sDE'%'C'
#'AB{}DE'.format('C')

In [ ]:
# 삼성전자 종목코드
sample_code = '005930'

In [ ]:
print(SNAP_URL%sample_code)

#Get PER data

In [ ]:
snap_url = SNAP_URL % sample_code # SNAP_URL에 formatting 한 것을 snap_url에 저장
snap_content = requests.get(snap_url).content # snap_url을 가져옴
snap_tree = html.fromstring(snap_content) #  

In [ ]:
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[2]/dd')[0].text 
per = float(per) # 실수 형태로 저장
per

# Get Debt Ratio data

In [ ]:
# 부채비율 크롤링
ratio_url = RATIO_URL % sample_code
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)

In [ ]:
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)
debt_ratio

#Get All stock Code

In [ ]:
import pandas as pd

csv_url = '/content/drive/MyDrive/Colab Notebooks/company.csv'

company = pd.read_csv(csv_url)
print(company.shape)
company.head(10)

In [ ]:
#column명 바꾸기
#company.columns = ['Category', 'Company_name', 'code', 'Industry', 'Product', 'IPO', 'Month', 'CEO', 'Homepage', 'Local', 'Gcode']

In [ ]:
# company.종목코드
code_list = company['종목코드'] 
code_list

In [ ]:
sample_code_list = code_list[1000:1050]
sample_code_list

In [ ]:
def crawler(li) :
    
     # fomatting이 비어있는 snap_url, ratio_url을 만들어야 함
    SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
    RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'
    result = {} # result라는 비어있는 딕셔너리를 만듬
    for code in tqdm(li) : 
        try:
            #비어있는 딕셔너리 하나 만듬
            #get per
            snap_url = SNAP_URL % code 
            snap_content = requests.get(snap_url).content 
            snap_tree = html.fromstring(snap_content) 

            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[2]/dd')[0].text 
            per = float(per) 

            #get debt ratio
            ratio_url = RATIO_URL % code
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)

            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)

            #company name
            name = company[company['종목코드'] == code]['회사명'].values[0] 
            result[name] = [code, per, debt_ratio] #list안에 value 값

        except(TypeError, AttributeError, IndexError, ValueError) as err :
            # 에러가 났을 때 무시하고 다음으로 넘어감
            pass 
    result_df = pd.DataFrame(result)
    #전치 행렬
    result_df = result_df.transpose()

    result_df.columns = ['Code', 'PER', 'Dedt_ratio']

    return result_df

In [ ]:
result_df = crawler(sample_code_list)
result_df

In [ ]:
# copy 메서드
copy_df = result_df.copy()
copy_df

# Searching Stocks

In [ ]:
filtered_df = copy_df[(copy_df['PER'] <= 10) & (copy_df['PER'] > 0) & (copy_df['Dedt_ratio'] <= 50)]
filtered_df

In [ ]:
# 나중에 개수 늘려서 해보기
#오름차순으로 정렬
#ascending=False를 추가해주면 내림차순으로 변경
filtered_df.sort_values(by='PER', ascending=False).head(30)